In [1]:
from weighted_levenshtein import lev
import numpy as np
import codecs
import re
from collections import *
files = ['formy.txt', 'dramat.txt', 'popul.txt', 'proza.txt', 'publ.txt', 'wp.txt']
datasets_path = 'C:/Users/OptimusPrime/Desktop/Studia/NLP1/Natural-Language-Processing-1/Week 4/Data'

In [2]:
class OrderedCounter(Counter, OrderedDict):
    pass

In [3]:
substitute_costs = np.ones((128, 128), dtype=np.float64)
insert_costs = np.ones(128, dtype=np.float64)
delete_costs = np.ones(128, dtype=np.float64)
substitute_costs[ord('a'), ord('A')] = 0.001  # ą
substitute_costs[ord('c'), ord('C')] = 0.001  # ć
substitute_costs[ord('e'), ord('E')] = 0.001  # ę
substitute_costs[ord('l'), ord('L')] = 0.001  # ł
substitute_costs[ord('o'), ord('O')] = 0.001  # ó
substitute_costs[ord('n'), ord('N')] = 0.001  # ń
substitute_costs[ord('s'), ord('S')] = 0.001  # ś
substitute_costs[ord('z'), ord('X')] = 0.001  # ź
substitute_costs[ord('z'), ord('Z')] = 0.001  # ż

substitute_costs[ord('O'), ord('u')] = 0.001  # ó -> u
substitute_costs[ord('u'), ord('O')] = 0.001  # u -> ó
substitute_costs[ord('r'), ord('s')] = 0.001  # rz -> sz
substitute_costs[ord('s'), ord('r')] = 0.001  # sz -> rz
insert_costs[ord('r')] = 0.001                # ż -> rz
substitute_costs[ord('Z'), ord('z')] = 0.001  # ż -> z
delete_costs[ord('r')] = 0.001                # rz -> ż
insert_costs[ord('c')] = 0.001                # h -> ch
delete_costs[ord('c')] = 0.001                # ch -> h

In [5]:
all_words = list()
for file in files:
    with codecs.open(datasets_path + '/' + file, 'r', 'utf-8') as f:
        text = f.read().lower()
        text = re.sub(r"[,.*1-9:()\[\]!?;\'\"ú]", '', text)
        text = re.sub(r"ą", "A", text)
        text = re.sub(r"ć", "C", text)
        text = re.sub(r"ę", "E", text)
        text = re.sub(r"ł", "L", text)
        text = re.sub(r"ó", "O", text)
        text = re.sub(r"ń", "N", text)
        text = re.sub(r"ś", "S", text)
        text = re.sub(r"ź", "X", text)
        text = re.sub(r"ż", "Z", text)
        text = text.encode('utf-8')
        words = text.split()
        all_words += words
vocabulary = OrderedCounter(all_words)

In [6]:
wrong_word = 'gzegzolka'
ww = re.sub(r"ą", "A", wrong_word)
ww = re.sub(r"ć", "C", ww)
ww = re.sub(r"ę", "E", ww)
ww = re.sub(r"ł", "L", ww)
ww = re.sub(r"ó", "O", ww)
ww = re.sub(r"ń", "N", ww)
ww = re.sub(r"ś", "S", ww)
ww = re.sub(r"ź", "X", ww)
ww = re.sub(r"ż", "Z", ww)

In [7]:
lev_met = [lev(ww, 
               word, 
               insert_costs=insert_costs, 
               substitute_costs=substitute_costs, 
               delete_costs=delete_costs) for word in vocabulary]
vocab_list = list(vocabulary.keys())
lev_met, vocab_list = (list(t) for t in zip(*sorted(zip(lev_met, vocab_list))))

lev_met = lev_met[:20]
vocab_list = vocab_list[:20]
print("Words with the smallest levenstein metric value:")
for word, met in zip(vocab_list, lev_met):
    print(word, met)

Words with the smallest levenstein metric value:
b'gZegZOLka' 0.004
b'gZegZOLkA' 0.005
b'gZegZOLkE' 1.004
b'gZegZOLki' 1.004
b'gZegZOLko' 1.004
b'gZegZOLkach' 1.005
b'gZegZOLce' 2.004
b'gZegZOLek' 2.004
b'gZegZOLkami' 2.004
b'gZegZOLkom' 2.004
b'czeczotka' 3.0
b'czeczotkA' 3.001
b'gruczoLka' 3.001
b'grzechotka' 3.001
b'grzegorza' 3.001
b'grzesznika' 3.001
b'rzeczuLka' 3.001
b'brzegOwka' 3.002
b'grzechotkA' 3.002
b'grzeszyLa' 3.002


In [8]:
maximum = lev_met[-1]
P_wc = [1 - met / maximum for met in lev_met]
print("Pwc:")
for word, prob in zip(vocab_list, P_wc):
    print(word, prob)

Pwc:
b'gZegZOLka' 0.9986675549633578
b'gZegZOLkA' 0.9983344437041972
b'gZegZOLkE' 0.6655562958027981
b'gZegZOLki' 0.6655562958027981
b'gZegZOLko' 0.6655562958027981
b'gZegZOLkach' 0.6652231845436376
b'gZegZOLce' 0.33244503664223846
b'gZegZOLek' 0.33244503664223846
b'gZegZOLkami' 0.33244503664223846
b'gZegZOLkom' 0.33244503664223846
b'czeczotka' 0.0006662225183210468
b'czeczotkA' 0.0003331112591604679
b'gruczoLka' 0.0003331112591604679
b'grzechotka' 0.0003331112591604679
b'grzegorza' 0.0003331112591604679
b'grzesznika' 0.0003331112591604679
b'rzeczuLka' 0.0003331112591604679
b'brzegOwka' 0.0
b'grzechotkA' 0.0
b'grzeszyLa' 0.0


In [9]:
number_of_occurences = [vocabulary[word] for word in vocab_list]
max_number_of_occurences = max(number_of_occurences)
print("Number of occurences of particular word:")
for word, numb in zip(vocab_list, number_of_occurences):
    print(word, numb)

Number of occurences of particular word:
b'gZegZOLka' 2
b'gZegZOLkA' 1
b'gZegZOLkE' 1
b'gZegZOLki' 2
b'gZegZOLko' 1
b'gZegZOLkach' 1
b'gZegZOLce' 1
b'gZegZOLek' 1
b'gZegZOLkami' 1
b'gZegZOLkom' 1
b'czeczotka' 2
b'czeczotkA' 1
b'gruczoLka' 1
b'grzechotka' 1
b'grzegorza' 3
b'grzesznika' 1
b'rzeczuLka' 1
b'brzegOwka' 1
b'grzechotkA' 1
b'grzeszyLa' 1


In [10]:
P_c = [numb/max_number_of_occurences for numb in number_of_occurences]
print("Pc:")
for word, prob in zip(vocab_list, P_c):
    print(word, prob)

Pc:
b'gZegZOLka' 0.6666666666666666
b'gZegZOLkA' 0.3333333333333333
b'gZegZOLkE' 0.3333333333333333
b'gZegZOLki' 0.6666666666666666
b'gZegZOLko' 0.3333333333333333
b'gZegZOLkach' 0.3333333333333333
b'gZegZOLce' 0.3333333333333333
b'gZegZOLek' 0.3333333333333333
b'gZegZOLkami' 0.3333333333333333
b'gZegZOLkom' 0.3333333333333333
b'czeczotka' 0.6666666666666666
b'czeczotkA' 0.3333333333333333
b'gruczoLka' 0.3333333333333333
b'grzechotka' 0.3333333333333333
b'grzegorza' 1.0
b'grzesznika' 0.3333333333333333
b'rzeczuLka' 0.3333333333333333
b'brzegOwka' 0.3333333333333333
b'grzechotkA' 0.3333333333333333
b'grzeszyLa' 0.3333333333333333


In [42]:
P_cw = [ probs[0]  * probs[1] for probs in zip(P_wc, P_c)]
P_cw, sorted_hints = (list(t) for t in zip(*sorted(zip(P_cw, vocab_list), reverse=True)))
print("Pcw:")
for word, prob in zip(sorted_hints, P_cw):
    print(word, prob)

b'gZegZOLka' 0.6657783699755718
b'gZegZOLki' 0.4437041972018654
b'gZegZOLkA' 0.33277814790139904
b'gZegZOLko' 0.2218520986009327
b'gZegZOLkE' 0.2218520986009327
b'gZegZOLkach' 0.22174106151454587
b'gZegZOLkom' 0.11081501221407948
b'gZegZOLkami' 0.11081501221407948
b'gZegZOLek' 0.11081501221407948
b'gZegZOLce' 0.11081501221407948
b'czeczotka' 0.00044414834554736454
b'grzegorza' 0.0003331112591604679
b'rzeczuLka' 0.00011103708638682264
b'grzesznika' 0.00011103708638682264
b'grzechotka' 0.00011103708638682264
b'gruczoLka' 0.00011103708638682264
b'czeczotkA' 0.00011103708638682264
b'grzeszyLa' 0.0
b'grzechotkA' 0.0
b'brzegOwka' 0.0
